In [2]:
import os
import pandas as pd

In [4]:
data_path = "data"

In [5]:
ts_sw_df = pd.read_csv(
    os.path.join(
        data_path, "time-series_surface-water_quality.csv"
    )
)
flow_df = pd.read_csv(
    os.path.join(
        data_path, "time-series_surface-water_flow.csv"
    )
)